In [6]:
import boto3
import configparser
import json
import pandas as pd

In [7]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

CLUSTER_TYPE           = config.get("CLUSTER","CLUSTER_TYPE")
CLUSTER_NUM_NODES      = config.get("CLUSTER","CLUSTER_NUM_NODES")
CLUSTER_NODE_TYPE      = config.get("CLUSTER","CLUSTER_NODE_TYPE")
CLUSTER_IDENTIFIER     = config.get("CLUSTER","CLUSTER_IDENTIFIER")

DB_NAME                = config.get("DB","DB_NAME")
DB_USER                = config.get("DB","DB_USER")
DB_PASSWORD            = config.get("DB","DB_PASSWORD")
DB_PORT                = config.get("DB","DB_PORT")

IAM_ROLE_NAME          = config.get("IAM", "IAM_ROLE_NAME")

BUCKET_NAME            = config.get("S3", "BUCKET_NAME")
LOG_DATA               = config.get("S3", "LOG_DATA")
SONG_DATA              = config.get("S3", "SONG_DATA")
LOG_JSONPATH           = config.get("S3", "LOG_JSONPATH")

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "CLUSTER_NUM_NODES", "CLUSTER_NODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT", "IAM_ROLE_NAME", "LOG_DATA", "SONG_DATA"],
              "Value":
                  [CLUSTER_TYPE, CLUSTER_NUM_NODES, CLUSTER_NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, IAM_ROLE_NAME, LOG_DATA, SONG_DATA]
             })

,Param,Value
0,CLUSTER_TYPE,multi-node
1,CLUSTER_NUM_NODES,1
2,CLUSTER_NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,redshift-cluster
4,DB_NAME,music_db
5,DB_USER,awsuser
6,DB_PASSWORD,Passw0rd
7,DB_PORT,5439
8,IAM_ROLE_NAME,RedshiftS3Role
9,LOG_DATA,s3://udacity-dend/log_data


In [8]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

sampleDbBucket =  s3.Bucket(BUCKET_NAME)

for obj in sampleDbBucket.objects.filter(Prefix=LOG_DATA.split('/')[-1]):
    print(obj.key)

log_data/
log_data/2018/11/2018-11-01-events.json
log_data/2018/11/2018-11-02-events.json
log_data/2018/11/2018-11-03-events.json
log_data/2018/11/2018-11-04-events.json
log_data/2018/11/2018-11-05-events.json
log_data/2018/11/2018-11-06-events.json
log_data/2018/11/2018-11-07-events.json
log_data/2018/11/2018-11-08-events.json
log_data/2018/11/2018-11-09-events.json
log_data/2018/11/2018-11-10-events.json
log_data/2018/11/2018-11-11-events.json
log_data/2018/11/2018-11-12-events.json
log_data/2018/11/2018-11-13-events.json
log_data/2018/11/2018-11-14-events.json
log_data/2018/11/2018-11-15-events.json
log_data/2018/11/2018-11-16-events.json
log_data/2018/11/2018-11-17-events.json
log_data/2018/11/2018-11-18-events.json
log_data/2018/11/2018-11-19-events.json
log_data/2018/11/2018-11-20-events.json
log_data/2018/11/2018-11-21-events.json
log_data/2018/11/2018-11-22-events.json
log_data/2018/11/2018-11-23-events.json
log_data/2018/11/2018-11-24-events.json
log_data/2018/11/2018-11-25-ev

In [9]:
s3_client = boto3.client('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

In [14]:
content_object = s3_client.get_object(Bucket=BUCKET_NAME, Key='song_data/A/T/R/TRATRGV128E0785AFB.json')
file_content = content_object['Body'].read()#.decode('utf-8')
json_content = json.loads(file_content)
print(json_content)
print(file_content)

{'artist_id': 'ARLKCMN1187FB599FB', 'artist_latitude': 36.16778, 'artist_location': 'Nashville', 'artist_longitude': -86.77836, 'artist_name': 'Big & Rich', 'duration': 173.40036, 'num_songs': 1, 'song_id': 'SOSEAXX12A6701F113', 'title': "Drinkin' 'Bout You (Album Version)", 'year': 2004}
b'{"artist_id":"ARLKCMN1187FB599FB","artist_latitude":36.16778,"artist_location":"Nashville","artist_longitude":-86.77836,"artist_name":"Big & Rich","duration":173.40036,"num_songs":1,"song_id":"SOSEAXX12A6701F113","title":"Drinkin\' \'Bout You (Album Version)","year":2004}'


In [20]:
import json
pd.DataFrame(json.loads(file_content),index=[0])

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARLKCMN1187FB599FB,36.16778,Nashville,-86.77836,Big & Rich,173.40036,1,SOSEAXX12A6701F113,Drinkin' 'Bout You (Album Version),2004


In [30]:


content_object = s3_client.get_object(Bucket=BUCKET_NAME, Key='log_json_path.json')
file_content = content_object['Body'].read()#.decode('utf-8')
json_content = json.loads(file_content)
print(json_content)


{'jsonpaths': ["$['artist']", "$['auth']", "$['firstName']", "$['gender']", "$['itemInSession']", "$['lastName']", "$['length']", "$['level']", "$['location']", "$['method']", "$['page']", "$['registration']", "$['sessionId']", "$['song']", "$['status']", "$['ts']", "$['userAgent']", "$['userId']"]}


In [31]:


content_object = s3_client.get_object(Bucket=BUCKET_NAME, Key='log_data/2018/11/2018-11-18-events.json')
file_content = content_object['Body'].read().decode('utf-8')

for split in file_content.split('\n'):
    json_content = json.loads(split)
    print(pd.DataFrame(json_content,index=[0]))
    break

            artist       auth firstName gender  itemInSession lastName  \
0  Rokia TraorÃÂ©  Logged In   Stefany      F              0    White   

      length level     location method      page  registration  sessionId  \
0  274.88608  free  Lubbock, TX    PUT  NextSong  1.540708e+12        693   

  song  status             ts  \
0  Zen     200  1542508401796   

                                           userAgent userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     83  


In [32]:


content_object = s3_client.get_object(Bucket=BUCKET_NAME, Key='log_data/2018/11/2018-11-18-events.json')
file_content = content_object['Body'].read().decode('utf-8')

file_content

'{"artist":"Rokia Traor\\u00c3\\u0083\\u00c2\\u00a9","auth":"Logged In","firstName":"Stefany","gender":"F","itemInSession":0,"lastName":"White","length":274.88608,"level":"free","location":"Lubbock, TX","method":"PUT","page":"NextSong","registration":1540708070796.0,"sessionId":693,"song":"Zen","status":200,"ts":1542508401796,"userAgent":"\\"Mozilla\\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\\/537.36 (KHTML, like Gecko) Chrome\\/36.0.1985.143 Safari\\/537.36\\"","userId":"83"}\n{"artist":null,"auth":"Logged In","firstName":"Tucker","gender":"M","itemInSession":0,"lastName":"Garrison","length":null,"level":"free","location":"Oxnard-Thousand Oaks-Ventura, CA","method":"GET","page":"Home","registration":1540832693796.0,"sessionId":555,"song":null,"status":200,"ts":1542511280796,"userAgent":"\\"Mozilla\\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\\/537.77.4 (KHTML, like Gecko) Version\\/7.0.5 Safari\\/537.77.4\\"","userId":"40"}\n{"artist":"Camila","auth":"Logged In","fir

In [6]:
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [7]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:iam::119027528033:user/airflow_redshift_user is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::119027528033:role/RedshiftS3Role
1.2 Attaching Policy


ClientError: An error occurred (AccessDenied) when calling the AttachRolePolicy operation: User: arn:aws:iam::119027528033:user/airflow_redshift_user is not authorized to perform: iam:AttachRolePolicy on resource: role RedshiftS3Role

In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=CLUSTER_NODE_TYPE,
        NumberOfNodes=int(CLUSTER_NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

An error occurred (AccessDenied) when calling the CreateCluster operation: User: arn:aws:iam::119027528033:user/airflow_redshift_user is not authorized to perform: iam:PassRole on resource: arn:aws:iam::119027528033:role/myRedshiftRole
